In [1]:
import numba
import numpy
import concurrent.futures

Low rank approx

In [2]:
@numba.jit(nogil=True)
def low_rank_approx(x, k=10):
    u, s, v = numpy.linalg.svd(x)
    return numpy.dot(u[:, :k], numpy.dot(numpy.diag(s[:k]), v[:k, :]))


In [3]:
xs = numpy.random.random((30, 800, 600))

In [4]:
print(low_rank_approx(xs[0]).shape, xs[0].shape)
numpy.linalg.norm(low_rank_approx(xs[0])), numpy.linalg.norm(xs[0])

(800, 600) (800, 600)


(349.1584646167392, 399.81054759950013)

In [5]:
%%timeit
zs = [low_rank_approx(x) for x in xs]

1 loop, best of 3: 3.7 s per loop


In [6]:
%%timeit
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as exe:
    futs = exe.map(low_rank_approx, xs)
    zs = list(futs)

1 loop, best of 3: 3.58 s per loop


In [7]:
%%timeit
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as exe:
    futs = exe.map(low_rank_approx, xs)
    zs = list(futs)

1 loop, best of 3: 4.09 s per loop


In [8]:
%timeit low_rank_approx(xs[0])
%timeit low_rank_approx.py_func(xs[0])

10 loops, best of 3: 133 ms per loop
10 loops, best of 3: 134 ms per loop
